In [14]:
from isolation import Board
from sample_players import *
from game_agent import *

In [139]:
class MinimaxPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using depth-limited minimax
    search. You must finish and test this player to make sure it properly uses
    minimax to return a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        **************  YOU DO NOT NEED TO MODIFY THIS FUNCTION  *************

        For fixed-depth search, this function simply wraps the call to the
        minimax method, but this method provides a common interface for all
        Isolation agents, and you will replace it in the AlphaBetaPlayer with
        iterative deepening search.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.minimax(game, self.search_depth)

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move

    def minimax(self, game, depth):
        """Implement depth-limited minimax search algorithm as described in
        the lectures.

        This should be a modified version of MINIMAX-DECISION in the AIMA text.
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        moves = game.get_legal_moves(self)
        if depth > 0 and moves:
            move_scores = {move: self.minimax_score(game.forecast_move(move), depth - 1) for move in moves}

            return sorted(move_scores, key=lambda move: move_scores[move], reverse=True)[0]
        else: # we've lost or too deep
            return (-1, -1)

    def minimax_score(self, game, depth, maximize=True):
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        if depth == 0:
            return self.score(game, self)

        moves = game.get_legal_moves(self)
        if not moves:
            return self.score(game, self)
        else:
            move_scores = {move: self.minimax_score(game.forecast_move(move), depth - 1, not maximize) for move in moves}

            print("move scores: ", move_scores, " maximization: ", maximize, " sorted index 0: ",sorted(move_scores, key=lambda move: move_scores[move], reverse=True)[0])
            return move_scores[sorted(move_scores, key=lambda move: move_scores[move], reverse=True)[0]]

In [152]:
player1 = MinimaxPlayer(score_fn=open_move_score, timeout=10.0, search_depth=2)

player2 = GreedyPlayer()
game = Board(player1, player2, 8, 8)
game._board_state = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 42, 40]

player1.get_move(game, time_left=lambda: 10)

print(game.move_count)

move scores:  {(0, 7): 3.0, (3, 4): 3.0, (4, 5): 3.0, (1, 4): 3.0}  maximization:  True  sorted index 0:  (0, 7)
move scores:  {(3, 2): 3.0, (3, 4): 3.0, (0, 1): 3.0, (2, 1): 3.0}  maximization:  True  sorted index 0:  (3, 2)
move scores:  {(0, 3): 6.0, (1, 2): 6.0, (3, 6): 6.0, (1, 6): 6.0, (3, 2): 6.0, (4, 3): 6.0, (4, 5): 6.0}  maximization:  True  sorted index 0:  (0, 3)
move scores:  {(3, 6): -inf}  maximization:  True  sorted index 0:  (3, 6)
0


In [120]:
# create an isolation board (by default 7x7)
player1 = MinimaxPlayer(score_fn=open_move_score)
# player1 = GreedyPlayer()
player2 = GreedyPlayer()
game = Board(player1, player2, 5, 5)

# place player 1 on the board at row 2, column 3, then place player 2 on
# the board at row 0, column 5; display the resulting board state.  Note
# that the .apply_move() method changes the calling object in-place.
game.apply_move((2, 3))
game.apply_move((0, 5))
print(game.to_string())

# players take turns moving on the board, so player1 should be next to move
assert(player1 == game.active_player)

# get a list of the legal moves available to the active player
print(game.get_legal_moves())
# player1.minimax_score(game, 1)

# get a successor of the current state by making a copy of the board and
# applying a move. Notice that this does NOT change the calling object
# (unlike .apply_move()).
new_game = game.forecast_move((1, 1))
assert(new_game.to_string() != game.to_string())
print("\nOld state:\n{}".format(game.to_string()))
print("\nNew state:\n{}".format(new_game.to_string()))

# play the remainder of the game automatically -- outcome can be "illegal
# move", "timeout", or "forfeit"
winner, history, outcome = game.play()
print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
print(game.to_string())
print("Move history:\n{!s}".format(history))





     0   1   2   3   4
0  |   |   |   |   |   | 
1  |   |   |   |   |   | 
2  |   |   |   | 1 |   | 
3  |   |   |   |   |   | 
4  |   |   |   |   |   | 

[(4, 4), (3, 1), (0, 4), (0, 2), (1, 1), (4, 2)]

Old state:
     0   1   2   3   4
0  |   |   |   |   |   | 
1  |   |   |   |   |   | 
2  |   |   |   | 1 |   | 
3  |   |   |   |   |   | 
4  |   |   |   |   |   | 


New state:
     0   1   2   3   4
0  |   |   |   |   |   | 
1  |   | 1 |   |   |   | 
2  |   |   |   | - |   | 
3  |   |   |   |   |   | 
4  |   |   |   |   |   | 

move scores:  {(3, 2): -inf}  maximization:  False  sorted index 0:  (3, 2)
move scores:  {(2, 1): 5.0, (1, 0): 2.0}  maximization:  True  sorted index 0:  (2, 1)
move scores:  {(2, 1): 5.0, (1, 4): 2.0}  maximization:  True  sorted index 0:  (2, 1)
move scores:  {(1, 0): 2.0, (1, 4): 2.0}  maximization:  True  sorted index 0:  (1, 0)
move scores:  {(1, 4): 5.0, (1, 0): 5.0, (2, 1): 2.0}  maximization:  False  sorted index 0:  (1, 4)
move scores:  {(3, 0): 2.0,